### Trabalho 3 - Inversores
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado


O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.

   i. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   
   ii. Cada inversor é regido pelas seguintes transformações
$$\mathbf{invert}\mathtt(in,out)$$
$$x \gets \mathsf{read}(\mathtt{in})$$
$$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$            
$$\mathsf{write}(\mathtt{out},s)$$
   iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
   
   iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
   
   iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.
                    
   1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
   2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

### Implementação

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools

'''
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

'''
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['Inv_A'] = Symbol('A'+str(i),BVType(1))
    state['Inv_B'] = Symbol('B'+str(i),BVType(1))
    state['Inv_C'] = Symbol('C'+str(i),BVType(1))
    state['Inv_D'] = Symbol('D'+str(i),BVType(1))
    return state


### Criação do SFOTS

In [2]:
'''
t0 = And(Equals(curr['pc'],Int(0)), Equals(prox['pc'],Int(1)), Ite(TRUE(),
                                                                Equals(prox['Inv_B'], BVXor(curr['Inv_A'],curr['Inv_B'])),
                                                                Ite(Equals(curr['Inv_A'],BVOne(1)),
                                                                                Equals(prox['Inv_B'],BVZero(1)),
                                                                                Equals(prox['Inv_B'],BVOne(1)))))
'''
import random
# valor aleatorio entre 0 e 1 para determinar qual a funcao a usar |¬x ∥ s←s⊕x |
funcao = random.randint(0, 1)

def trans1(curr,prox):

    t0 = And(Equals(curr['pc'],Int(0)), Equals(prox['pc'],Int(1)), Equals(Int(funcao),Int(0)),
                                                                Ite(Equals(curr['Inv_A'],BVOne(1)),
                                                                                Equals(prox['Inv_B'],BVZero(1)),
                                                                                Equals(prox['Inv_B'],BVOne(1))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_C'],curr['Inv_C']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t1 = And(Equals(curr['pc'],Int(0)),Equals(prox['pc'],Int(1)), Equals(Int(funcao),Int(1)), 
             Equals(prox['Inv_B'],BVXor(curr['Inv_A'],curr['Inv_B'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_C'],curr['Inv_C']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t2 = And(Equals(curr['pc'],Int(1)),Equals(prox['pc'],Int(2)), Equals(Int(funcao),Int(0)), Ite(Equals(curr['Inv_B'],BVOne(1)),
                                                                                Equals(prox['Inv_C'],BVZero(1)),
                                                                                Equals(prox['Inv_C'],BVOne(1))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t3 = And(Equals(curr['pc'],Int(1)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],Int(2)), Equals(prox['Inv_C'],BVXor(curr['Inv_B'],curr['Inv_C'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t4 = And(Equals(curr['pc'],Int(2)), Equals(Int(funcao),Int(0)), Equals(prox['pc'],Int(3)), Ite(Equals(curr['Inv_C'],BVOne(1)),
                                                                                Equals(prox['Inv_D'],BVZero(1)),
                                                                                Equals(prox['Inv_D'],BVOne(1))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t5 = And(Equals(curr['pc'],Int(2)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],Int(3)), Equals(prox['Inv_D'],BVXor(curr['Inv_C'],curr['Inv_D'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t6 = And(Equals(curr['pc'],Int(3)), Equals(Int(funcao),Int(0)), Equals(prox['pc'],Int(0)), Ite(Equals(curr['Inv_D'],BVOne(1)),
                                                                                Equals(prox['Inv_A'],BVZero(1)),
                                                                                Equals(prox['Inv_A'],BVOne(1))),
            Equals(prox['Inv_D'],curr['Inv_D']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t7 = And(Equals(curr['pc'],Int(3)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],Int(0)), Equals(prox['Inv_A'],BVXor(curr['Inv_D'],curr['Inv_A'])),
             Equals(prox['Inv_D'],curr['Inv_D']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))

    return Or(t0,t1,t2,t3,t4,t5,t6,t7)
        
def init1(state):
    return And(Equals(state['pc'],Int(0)), 
          Equals(state['Inv_A'],BV(random.randint(0, 1),1)),
          Equals(state['Inv_B'],BV(random.randint(0, 1),1)),
          Equals(state['Inv_C'],BV(random.randint(0, 1),1)),
          Equals(state['Inv_D'],BV(random.randint(0, 1),1)),
           Not(And(Equals(state['Inv_A'],BVZero(1)),Equals(state['Inv_B'],BVZero(1)),Equals(state['Inv_C'],BVZero(1)),Equals(state['Inv_D'],BVZero(1))))
            )

def error1(state):
    return And(Equals(state['Inv_A'],Int(0)),
               Equals(state['Inv_B'],Int(0)),
               Equals(state['Inv_C'],Int(0)),
               Equals(state['Inv_D'],Int(0))
              )


In [3]:
def genTrace(vars,init,trans,error,n):
    with Solver(name="z3") as s:
        
        X = [declare(i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

print(funcao)
genTrace(['Inv_A','Inv_B','Inv_C','Inv_D','pc'],init1,trans1,error1,50)

0
Estado: 0
           pc = 0
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 0_1
           Inv_D = 1_1
Estado: 1
           pc = 1
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 0_1
           Inv_D = 1_1
Estado: 2
           pc = 2
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 1_1
Estado: 3
           pc = 3
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 0_1
Estado: 4
           pc = 0
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 0_1
Estado: 5
           pc = 1
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 0_1
Estado: 6
           pc = 2
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 0_1
Estado: 7
           pc = 3
           Inv_A = 1_1
           Inv_B = 0_1
           Inv_C = 1_1
           Inv_D = 0_1
Estado: 8
           pc = 0
          

###  Model checking com interpolantes

In [4]:
def invert(trans):
    return (lambda c, p: trans(p,c))